<a href="https://colab.research.google.com/github/ThanuMahee12/ayush-vision/blob/data-prepare/DataSet/Prepare/DataSetPrepareInGoogle_Account.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Colab

In [1]:
from google.colab import auth

In [2]:
from googleapiclient.discovery import build

In [3]:
from google.colab import drive

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
firebase_key='/content/drive/MyDrive/ServiceKey.json'

In [7]:
firebase_count=1

### user

In [8]:
auth.authenticate_user()
service = build('oauth2', 'v2')
user_info = service.userinfo().get().execute()
email = user_info['email']
profile=user_info['picture']


## Firebase

In [ ]:
!pip install firebase-admin

In [9]:
import firebase_admin

In [10]:
from firebase_admin import credentials,storage


In [11]:
from firebase_admin import firestore

In [12]:

if firebase_count==1:
  cred = credentials.Certificate(firebase_key)
  firebase_admin.initialize_app(cred,{
    'storageBucket': 'ayush-vision-asw4gh.appspot.com'
})
firebase_count=firebase_count+1

In [13]:
db = firestore.client()

In [14]:
bucket = storage.bucket()

In [15]:
dataset_col = db.collection("dataset")

### Default

In [16]:
import numpy as np
import os
from datetime import date,datetime
import json
import csv
import yaml

In [17]:
def count_images_in_folder(folder_path):
    return sum([len(files) for r, d, files in os.walk(folder_path) if any(file.endswith(('.jpg', '.jpeg', '.png')) for file in files)])

def dictory_images_count(root_dir):
  train_images = count_images_in_folder(os.path.join(root_dir, 'train','images'))
  val_images = count_images_in_folder(os.path.join(root_dir, 'valid','images'))
  test_images = count_images_in_folder(os.path.join(root_dir, 'test','images'))
  total_images = train_images + val_images + test_images
  return {
      'total_count':total_images,
      'val_count':val_images,
      'train_count':train_images,
      'test_count':test_images
  }

## Yolo

In [5]:
Dataset_dir='/content/drive/MyDrive/AyushVision/yolo/dataset'

In [18]:
yolo_doc=dataset_col.document('yolo')
doc=yolo_doc.get()

In [19]:
if not doc.exists:
  doc_ref= db.collection("dataset").document('yolo')
  doc_ref.set({'algorithum':'yolo','version':'1','mode':'detection','url':'yolo'})
else:
  doc_ref= yolo_doc

In [28]:
import os
import yaml
from collections import defaultdict

def load_class_names(yaml_path):
    """Load class names from data.yaml."""
    with open(yaml_path, 'r') as file:
        data = yaml.safe_load(file)
    return {i: value for i, value in enumerate(data['names'])}

def map_files_to_classes(base_path, yaml_path):
    """Map number of files for each class."""
    class_names = load_class_names(yaml_path)

    # Initialize data structures
    class_files = defaultdict(lambda: {
        'count': 0,
        'files': []
    })

    # Paths to label files
    label_paths = {
        'train': os.path.join(base_path, 'train', 'labels'),
        'test': os.path.join(base_path, 'test', 'labels'),
        'valid': os.path.join(base_path, 'valid', 'labels')
    }

    # Process each label file
    for dataset in ['train', 'test', 'valid']:
        label_path = label_paths[dataset]
        for filename in os.listdir(label_path):
            if filename.endswith('.txt'):
                image_file = filename.replace('.txt', '.jpg')  # Assuming images are .jpg
                label_file = os.path.join(label_path, filename)

                with open(label_file, 'r') as file:
                    classes = set()
                    for line in file:
                        parts = line.strip().split()
                        class_id = int(parts[0])
                        classes.add(class_names.get(class_id, 'unknown'))

                for cls in classes:
                    class_files[cls]['count'] += 1
                    class_files[cls]['files'].append({
                        'imagefile': image_file,
                        'annotationfile': filename,
                        'classes': list(classes)
                    })

    return class_files

# Example usage



#### Yolo Leaves

In [ ]:
yolo_ref_leaves=doc_ref.collection("Leaves")

In [ ]:
yolo_doc_leave_basic=yolo_ref_leaves.document('basic')

In [ ]:
yolo_doc_leave_basic.set({
    'account':email,
    'profile':profile,
})

update_time {
  seconds: 1725862376
  nanos: 280581000
}

In [ ]:
intial_basic_col=yolo_doc_leave_basic.collection("Intial")

In [ ]:
subfolders=os.listdir(Dataset_dir)

In [ ]:
total_datasetCount=0
names=[]

In [ ]:
for sub in subfolders:
  yaml_file=os.path.join(Dataset_dir,sub,'data.yaml')
  sub_doc=intial_basic_col.document(sub)
  sub_doc_col_util=sub_doc.collection('Util')
  sub_doc_col_util_dataYaml=sub_doc_col_util.document('datYaml')
  sub_doc_col_util_Dataset_readMe=sub_doc_col_util.document('readme_Dataset')
  sub_doc_col_util_roboflow_readMe=sub_doc_col_util.document('readme_roboflow')
  dataset_count=dictory_images_count(os.path.join(Dataset_dir,sub))
  total_datasetCount=total_datasetCount+dataset_count['total_count']
  sub_doc.set(dataset_count)
  datasetLines,roboflowlines=[],[]
  with open(os.path.join(Dataset_dir,sub,'README.dataset.txt'), "r") as file:
    datasetLines = file.readlines()
  sub_doc_col_util_Dataset_readMe.set({'dataset':datasetLines})
  with open(os.path.join(Dataset_dir,sub,'README.roboflow.txt'), "r") as file:
    roboflowlines = file.readlines()
  sub_doc_col_util_roboflow_readMe.set({
      'roboflow':roboflowlines
  })
  with open(yaml_file, 'r') as file:
    yaml_content = yaml.safe_load(file)
    names.extend(yaml_content['names'])
    sub_doc_col_util_dataYaml.set(yaml_content)
    sub_doc.update({'names':yaml_content['names'],'no_of_species':yaml_content['nc'],'roboflow':yaml_content['roboflow']})

In [ ]:
yolo_doc_leave_basic.update({
    'count':total_datasetCount,
    'names':list(set(names))
})

update_time {
  seconds: 1725862570
  nanos: 231251000
}

In [ ]:
root_path=Dataset_dir
for folder in os.listdir(root_path):
  base_path =os.path.join(root_path,folder)
  yaml_path = os.path.join(base_path,'data.yaml')
  class_files = map_files_to_classes(base_path, yaml_path)
  sub_doc=intial_basic_col.document(folder)
  sub_doc_collection=sub_doc.collection("Species")
  for cls, info in class_files.items():
    doc=sub_doc_collection.document(cls)
    doc.set({
        'count':info['count'],
        'file_info':info['files']
    })

NameError: name 'Dataset_dir' is not defined

### Prepare

In [ ]:
intial_prepare_col=yolo_doc_leave_basic.collection("Prepare")

In [ ]:
Dataset_dir="/content/drive/MyDrive/AyushVision/yolo/dataset/final"
yaml_file=os.path.join(Dataset_dir,sub,'data.yaml')
sub_doc=intial_prepare_col.document("final")
sub_doc_col_util=sub_doc.collection('Util')
sub_doc_col_util_dataYaml=sub_doc_col_util.document('datYaml')
dataset_count=dictory_images_count(os.path.join(Dataset_dir,sub))
total_datasetCount=total_datasetCount+dataset_count['total_count']
sub_doc.set(dataset_count)
with open(yaml_file, 'r') as file:
    yaml_content = yaml.safe_load(file)
    names.extend(yaml_content['names'])
    sub_doc_col_util_dataYaml.set(yaml_content)
    sub_doc.update({'names':yaml_content['names'],'no_of_species':yaml_content['nc'],'roboflow':yaml_content['roboflow']})

# Tumber

In [20]:
yolo_ref_tumbers=doc_ref.collection("Tumbers")

In [22]:
yolo_doc_tumber_basic=yolo_ref_tumbers.document('basic')

In [23]:
yolo_doc_tumber_basic.set({
    'account':email,
    'profile':profile,
})

update_time {
  seconds: 1726247920
  nanos: 666678000
}

In [24]:
intial_basic_col=yolo_doc_tumber_basic.collection("Intial")

In [25]:
subfolders=os.listdir(Dataset_dir)

In [26]:
total_datasetCount=0
names=[]

In [27]:
for sub in subfolders:
  yaml_file=os.path.join(Dataset_dir,sub,'data.yaml')
  sub_doc=intial_basic_col.document(sub)
  sub_doc_col_util=sub_doc.collection('Util')
  sub_doc_col_util_dataYaml=sub_doc_col_util.document('datYaml')
  sub_doc_col_util_Dataset_readMe=sub_doc_col_util.document('readme_Dataset')
  sub_doc_col_util_roboflow_readMe=sub_doc_col_util.document('readme_roboflow')
  dataset_count=dictory_images_count(os.path.join(Dataset_dir,sub))
  total_datasetCount=total_datasetCount+dataset_count['total_count']
  sub_doc.set(dataset_count)
  datasetLines,roboflowlines=[],[]
  with open(os.path.join(Dataset_dir,sub,'README.dataset.txt'), "r") as file:
    datasetLines = file.readlines()
  sub_doc_col_util_Dataset_readMe.set({'dataset':datasetLines})
  with open(os.path.join(Dataset_dir,sub,'README.roboflow.txt'), "r") as file:
    roboflowlines = file.readlines()
  sub_doc_col_util_roboflow_readMe.set({
      'roboflow':roboflowlines
  })
  with open(yaml_file, 'r') as file:
    yaml_content = yaml.safe_load(file)
    names.extend(yaml_content['names'])
    sub_doc_col_util_dataYaml.set(yaml_content)
    sub_doc.update({'names':yaml_content['names'],'no_of_species':yaml_content['nc'],'roboflow':yaml_content['roboflow']})

In [29]:
yolo_doc_tumber_basic.update({
    'count':total_datasetCount,
    'names':list(set(names))
})

update_time {
  seconds: 1726248158
  nanos: 750474000
}

In [30]:
root_path=Dataset_dir
for folder in os.listdir(root_path):
  base_path =os.path.join(root_path,folder)
  yaml_path = os.path.join(base_path,'data.yaml')
  class_files = map_files_to_classes(base_path, yaml_path)
  sub_doc=intial_basic_col.document(folder)
  sub_doc_collection=sub_doc.collection("Species")
  for cls, info in class_files.items():
    doc=sub_doc_collection.document(cls)
    doc.set({
        'count':info['count'],
        'file_info':info['files']
    })

## Prepare

In [31]:
intial_prepare_col=yolo_doc_tumber_basic.collection("Prepare")

In [33]:
Dataset_dir="/content/drive/MyDrive/AyushVision/yolo/dataset/final"
yaml_file=os.path.join(Dataset_dir,'data.yaml')
sub_doc=intial_prepare_col.document("final")
sub_doc_col_util=sub_doc.collection('Util')
sub_doc_col_util_dataYaml=sub_doc_col_util.document('datYaml')
dataset_count=dictory_images_count(os.path.join(Dataset_dir,sub))
total_datasetCount=total_datasetCount+dataset_count['total_count']
sub_doc.set(dataset_count)
with open(yaml_file, 'r') as file:
    yaml_content = yaml.safe_load(file)
    names.extend(yaml_content['names'])
    sub_doc_col_util_dataYaml.set(yaml_content)
    sub_doc.update({'names':yaml_content['names'],'no_of_species':yaml_content['nc'],'roboflow':yaml_content['roboflow']})

In [34]:
base_path =Dataset_dir
yaml_path = os.path.join(base_path,'data.yaml')
class_files = map_files_to_classes(base_path, yaml_path)
sub_doc=intial_prepare_col.document('final')
sub_doc_collection=sub_doc.collection("Species")
for cls, info in class_files.items():
    doc=sub_doc_collection.document(cls)
    doc.set({
        'count':info['count'],
        'file_info':info['files']
    })